# Домашнее задание по теме  
## «Алгоритм Random Forest» 

Импорт библиотек

In [30]:
# Импорт библиотек
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# import joblib

## Задача 1 Классификация

In [ ]:
# Загрузка данных
# Использую обработанные данные из задачи классификация SVM
archive_path = 'input_data/input_data.zip'
with zipfile.ZipFile(archive_path, 'r') as archive:
 
    with archive.open('train_class.csv') as file:
        df_class = pd.read_csv(file)
df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38984 entries, 0 to 38983
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  38984 non-null  int64  
 1   height_cm            38984 non-null  int64  
 2   weight_kg            38984 non-null  int64  
 3   waist_cm             38984 non-null  float64
 4   eyesight_left        38984 non-null  float64
 5   eyesight_right       38984 non-null  float64
 6   hearing_left         38984 non-null  int64  
 7   hearing_right        38984 non-null  int64  
 8   systolic             38984 non-null  int64  
 9   relaxation           38984 non-null  int64  
 10  fasting_blood_sugar  38984 non-null  int64  
 11  cholesterol          38984 non-null  int64  
 12  triglyceride         38984 non-null  int64  
 13  hdl                  38984 non-null  int64  
 14  ldl                  38984 non-null  int64  
 15  hemoglobin           38984 non-null 

In [18]:
# Разделение данных на обучающую и тестовую выборки
X = df_class.drop('smoking', axis=1)  # Признаки
y = df_class['smoking']  # Целевая переменная

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
# Определяем сетку гиперпараметров
param_grid = {
    'n_estimators': [100, 300, 500],  # Количество деревьев
    'max_depth': [None, 10, 20, 30],  # Максимальная глубина дерева
    'min_samples_split': [2, 5, 10],  # Минимальное количество образцов для разделения
    'min_samples_leaf': [1, 2, 4],    # Минимальное количество образцов в листе
    'max_features': ['sqrt', 'log2']  # Количество признаков для поиска лучшего разделения
}

# Создаем модель
model = RandomForestClassifier(random_state=42)

# Инициализируем GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  # Метрика для оптимизации
    cv=5,                # Количество фолдов для кросс-валидации
    n_jobs=-1,           # Использовать все доступные ядра процессора
    verbose=2            # Вывод подробной информации о процессе
)

# Запускаем поиск по сетке
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Лучшая модель
best_model = grid_search.best_estimator_

# Оценка качества на тестовой выборке
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy на тестовой выборке: {accuracy:.2f}')

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Accuracy на тестовой выборке: 0.80


In [20]:
model = grid_search.best_estimator_
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

In [21]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.80


In [22]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      7412
           1       0.72      0.73      0.73      4284

    accuracy                           0.80     11696
   macro avg       0.78      0.78      0.78     11696
weighted avg       0.80      0.80      0.80     11696

[[6219 1193]
 [1168 3116]]


Вывод:
- Гиперпараметры подобраны с помощью GridSearchCV. 'max_depth' = 20, 'max_features' = 'sqrt', 'min_samples_leaf' = 1, 'min_samples_split' = 2, 'n_estimators'= 500
- Модель правильно предсказала 80% всех объектов, но нужно учитывать что предсказания класса 1 73% из-за дисбаланса классов в df

## Задача 2. Регрессия

In [ ]:
# Загрузка данных
# Использую обработанные данные из задачи регрессия SVM 
archive_path = 'input_data/input_data.zip'
with zipfile.ZipFile(archive_path, 'r') as archive:
 
    with archive.open('train_regr.csv') as file:
        df_regr = pd.read_csv(file)
df_regr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11520 entries, 0 to 11519
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   11520 non-null  float64
 1   annual_income         11520 non-null  float64
 2   number_of_dependents  11520 non-null  float64
 3   health_score          11520 non-null  float64
 4   previous_claims       11520 non-null  float64
 5   vehicle_age           11520 non-null  float64
 6   credit_score          11520 non-null  float64
 7   insurance_duration    11520 non-null  float64
 8   premium_amount        11520 non-null  float64
 9   gender_Male           11520 non-null  int64  
 10  smoking_status_Yes    11520 non-null  int64  
 11  location_Suburban     11520 non-null  int64  
 12  location_Urban        11520 non-null  int64  
 13  gender_Male.1         11520 non-null  int64  
dtypes: float64(9), int64(5)
memory usage: 1.2 MB


In [24]:
# Загрузка данных
X = df_regr.drop('premium_amount', axis=1)  # Признаки
y = df_regr['premium_amount']  # Целевая переменная

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Определяем сетку гиперпараметров
param_grid = {
    'n_estimators': [100, 300, 500],  # Количество деревьев
    'max_depth': [None, 10, 20, 30],   # Максимальная глубина дерева
    'min_samples_split': [2, 5, 10],   # Минимальное количество образцов для разделения
    'min_samples_leaf': [1, 2, 4],     # Минимальное количество образцов в листе
    'max_features': ['sqrt', 'log2']   # Количество признаков для поиска лучшего разделения
}

# Создаем модель
model = RandomForestRegressor(random_state=42)

# Инициализируем GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Метрика для оптимизации (отрицательная MSE)
    cv=5,                              # Количество фолдов для кросс-валидации
    n_jobs=-1,                         # Использовать все доступные ядра процессора
    verbose=2                          # Вывод подробной информации о процессе
)

# Запускаем поиск по сетке
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Лучшая модель
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Лучшие параметры: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1000}


In [40]:
# Обучаем лучшую модель на всей обучающей выборке
best_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=1000, random_state=42)

In [41]:
# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R²: {r2:.2f}')

MSE: 607364.06
MAE: 623.63
R²: 0.02


Выводы:
- MSE (Mean Squared Error) = 607364.06 Очень высокое значение MSE указывает на то, что модель делает большие ошибки в предсказаниях.
- MAE (Mean Absolute Error) = 623.63 Высокое значение MAE подтверждает, что модель ошибается в предсказаниях.
- R² (Коэффициент детерминации) = 0.02 Значение R² близко к 0, что означает, что модель практически не объясняет целевую переменную.
- R² немного лучше ранее обученных моделей (максимальный был у KNN = 0.005), но попрежнему очень низок.
- Модель работает не лучше, чем простое предсказание среднего значения.